## 1. 内容和目标 
在本示例中，我们将介绍和演示以下内容:
- Uni-Mol的主要特点 
- Uni-Mol的分子、原子表示方法 
- Uni-Mol的核心模块和相关参数 
- 如何微调超参数（如学习率 learning rate）来训练模型 
- 实际应用 

## 2. 挂载数据集路径 
本教学案例需要使用的数据，已通过Bohrium数据集功能共享给当前项目的用户。

数据集的路径为

In [ ]:
Dataset_Dir = '/bohr/data-ksoi/v1/data/' 

## 2.1 读取数据，并对数据进行拆分和初步的分析 
【注】Uni-Mol进行Finetuning时，默认使用<i>5-Fold Cross Validation</i>的方式进行模型验证。 

In [ ]:
import os 
import pandas as pd

# 读入数据 
train_data_full = pd.read_csv(os.path.join(Dataset_Dir, 'mol_train.csv'))
train_data_full.columns=["SMILES", "TARGET"] 
train_data_full.to_csv("./mol_train_full.csv", index=False) 

train_data_split = train_data_full[:500] 
train_data_split.columns=["SMILES", "TARGET"]
train_data_split.to_csv("./mol_train_split.csv", index=False) 

valid_data_split = train_data_full[500:] 
valid_data_split.columns=["SMILES", "TARGET"]

directory = "/data/UniMol_QSAR"

# 如果目录不存在，则创建它
if not os.path.exists(directory):
    os.makedirs(directory)


valid_data_split.to_csv("/data/UniMol_QSAR/mol_valid_split.csv", index=False) 

test_data = pd.read_csv(os.path.join(Dataset_Dir, 'mol_test.csv'))
test_data.columns=["SMILES", "TARGET"]
test_data.to_csv("./mol_test.csv", index=False) 

## 4. Uni-Mol的分子、原子表示方法 

In [ ]:
# 导入Uni-Mol 
from unimol import UniMolRepr 
import numpy as np 

# single smiles unimol representation
clf = UniMolRepr(data_type='molecule', remove_hs=False) 

smiles_list = train_data_split["SMILES"].values.tolist()[:1] 

unimol_repr = clf.get_repr(smiles_list, return_atomic_reprs=True) 

# Uni-Mol分子表征, 使用cls token 
print(np.array(unimol_repr['cls_repr']).shape) 
print(np.array(unimol_repr['cls_repr'])) 

# Uni-Mol 原子级别表征，和rdkit Mol的原子顺序一致 
print(np.array(unimol_repr['atomic_reprs']).shape) 
print(np.array(unimol_repr['atomic_reprs'])) 

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from sklearn.decomposition import PCA

smiles_list = valid_data_split["SMILES"].values.tolist() 
y = valid_data_split["TARGET"].values.tolist() 
repr_dict = clf.get_repr(smiles_list)
unimol_repr_list = np.array(repr_dict['cls_repr']) 

pca = PCA(n_components=2) 
X_reduced = pca.fit_transform(unimol_repr_list) 

# 可视化
colors = ['r', 'g', 'b']
markers = ['s', 'o', 'D']
labels = ['Target:0','Target:1']

plt.figure(figsize=(8, 6))

for label, color, marker in zip(np.unique(y), colors, markers):
    plt.scatter(X_reduced[y == label, 0], 
                X_reduced[y == label, 1], 
                c=color, 
                marker=marker, 
                label=labels[label],
                edgecolors='black')

plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')
plt.legend(loc='best')
plt.title('Unimol Repr')
plt.show() 

## 5. Uni-Mol的核心模块和相关参数 

In [ ]:
# 导入Uni-Mol 
from unimol import MolTrain, MolPredict 

### 5.1. “Moltrain”模块的参数说明
#### task：选择对应的任务，目前支持五种任务类型：
- classification: 0/1分类
- regression: 回归
- multiclass: 多分类
- multilabel_classification: 多标签0/1分类
- multilabel_regression: 多标签回归
#### metrics: 对应需要模型优化的指标，传入的metrics可以用逗号分隔，为空默认，目前支持的指标如下：
- classification: auc,auprc,log_loss,f1_score, mcc,recall,precision,cohen_kappa;
- regression: mae, mse, rmse, r2, spearmanr;
- multiclass: log_loss, acc;
- multilabel_classification: log_loss, acc, auprc, cohen_kappa;
- multilabel_regression: mse, mae, rmse, r2;
#### data_type：输入的数据类型，目前仅支持molecule，后续会开放protein, crystal等更多数据源 
#### split：unimol 默认采用5-Fold Cross Validation验证方式，划分方式支持random和按照scaffold划分 
#### save_path：当前的任务路径，默认会覆盖文件 
#### epochs, learning_rate, batch_size, early_stopping：unimol训练时开放的超参数

### 5.2. 超参数调整（Hyperparameter Finetuning） 
超参数是机器学习模型中的参数，其值不能通过训练数据集直接学习得到，而是需要人为设定。例如，学习率、批处理大小、训练周期数、神经网络的层数和每层的节点数等都是典型的超参数。超参数调整的目的是为了找到一组能使模型在测试数据集上表现最好的超参数。因为超参数的值直接影响到模型的训练效果和预测性能，所以合理的调整超参数对于获得高质量的机器学习模型是非常重要的。

### 5.3. Metrics 
#### 5.3.1. F2-score 
`F2-score` 是 `F1-beta` 取 $beta = 2$ 时的特殊形式。  `F1-beta` 允许我们调整精确率（Precision）和召回率（Recall）之间的权重，其一般形式如下：

$$F1-beta = (1 + beta^2) * (Precision * Recall) / (beta^2 * Precision + Recall)$$

在本次赛事中，我们更在意模型的召回率，取 $beta = 2$。此时我们得到 `F2-score`，`F2-score` 的计算公式为：

$$F2-score = 5 * (Precision * Recall) / (4 * Precision + Recall)$$

其中，Precision = 真正例（TP）/（真正例（TP）+假正例（FP）），Recall = 真正例（TP）/（真正例（TP）+假负例（FN））。
TP（True Positive）表示实际为正例且预测为正例的样本数；TN（True Negative）表示实际为负例且预测为负例的样本数；FP（False Positive）表示实际为负例但预测为正例的样本数；FN（False Negative）表示实际为正例但预测为负例的样本数。

最终排行榜展示结果只保留 `F2-score` 的四位小数，并根据F2-score来评估模型的好坏：

- 接近1的F2-score表示模型性能很好，意味着模型在正确预测正例方面做得很好，同时尽量减少了假负例（即避免将实际为正的样本预测为负）。

- 接近0的F2-score表示模型性能很差，意味着模型在正确预测正例方面做得很差，同时产生了很多假负例。

#### 5.3.2. AUC-ROC 
- AUC-ROC是一种用于评估分类模型性能的指标，全称为Area Under the Receiver Operating Characteristic Curve（接收者操作特征曲线下的面积）。在这里，ROC曲线是一个描绘了真正类率（True Positive Rate, TPR）和假正类率（False Positive Rate, FPR）在不同分类阈值下变化情况的曲线。
- 真正类率（TPR）也叫敏感性，它度量的是所有实际为正的样本中，被正确地判断为正的比例。TPR = TP / (TP + FN)，其中TP是真正类的数量，FN是假负类的数量。假正类率（FPR）度量的是所有实际为负的样本中，被错误地判断为正的比例。FPR = FP / (FP + TN)，其中FP是假正类的数量，TN是真负类的数量。
- AUC值即为ROC曲线下的面积，它反映了模型在任意分类阈值下的整体性能。AUC值在0.5到1之间，值越接近1，模型的性能越好。一般来说，如果模型的AUC值大于0.5，那么模型就比随机猜测好；如果AUC值等于0.5，那么模型的性能就等同于随机猜测；如果AUC值小于0.5，那么模型的性能就比随机猜测差。
- AUC-ROC值的优点在于，它不依赖于特定的分类阈值，因此对于评估模型的整体性能非常有用，尤其是在正负样本不均衡的情况下。

【注】本次赛题采用 `F2-score` 作为评价指标。

In [ ]:
# 使用全量数据进行Fine-Tuneing，并使用Cross-Validation的方法验证模型的性能 
from sklearn.metrics import roc_auc_score, roc_curve, fbeta_score # 导入sklearn.metrics以输出模型的ROC-AUC和F2-score

threshold = 0.5 

train_full_results = {} 

font = {'family': 'serif',
        'color':  'black',
        'weight': 'normal',
        'size': 8} 

lr_ft = [1e-5, 1e-4, 1e-3] # 学习率
for i in range(len(lr_ft)): # 循环输入每个学习率   
    clf = MolTrain(task='classification', 
                   data_type='molecule',
                   epochs=20,
                   learning_rate=lr_ft[i],
                   batch_size=16,
                   early_stopping=5,
                   metrics='auc',
                   split='random',
                   save_path='./full_learning_rate_'+str(lr_ft[i]),
                  )
    
    clf.fit("./mol_train_full.csv") # 训练模型 
    
    cv_results = pd.DataFrame({'pred':clf.cv_pred.flatten(), 
                           'SMILES':train_data_full["SMILES"], 
                           'Target_BBB':train_data_full["TARGET"]}) 
     
    auc = roc_auc_score(cv_results.Target_BBB, cv_results.pred) 
    fpr, tpr, _ = roc_curve(cv_results.Target_BBB, cv_results.pred) 
    f2_score = fbeta_score(
        cv_results.Target_BBB, 
        [1 if p > threshold else 0 for p in cv_results.pred], 
        beta=2
    )
    
    train_full_results[f"Full Learning Rate: {lr_ft[i]}"] = {"AUC": auc, "FPR": fpr, "TPR": tpr, "F2_Score": f2_score} 
    print(f"[Learning Rate: {lr_ft[i]}]\tAUC:{auc:.4f}\tF2_Score:{f2_score:.4f}") 

sorted_train_full_results = sorted(train_full_results.items(), key=lambda x: x[1]["F2_Score"], reverse=True) # 根据AUC对结果进行排序

# 绘制ROC曲线
plt.figure(figsize=(10, 6), dpi=150)
plt.title("ROC Curves")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")

for name, result in sorted_train_full_results:
    if name.startswith("Full Learning Rate"):
        plt.plot(result["FPR"], result["TPR"], label=f"{name} (AUC:{result['AUC']:.4f} F2_Score:{result['F2_Score']:.4f})")

plt.legend(loc="lower right") 
plt.title("Train_Full_Model", fontdict=font) 
plt.show() 

In [ ]:
# # 使用手动拆分的数据train_data_split进行Fine-Tuneing 
# lr_ft = [1e-5, 1e-4, 1e-3] # 学习率
# for i in range(len(lr_ft)): # 循环输入每个学习率   
#     clf = MolTrain(task='classification', 
#                    data_type='molecule',
#                    epochs=20,
#                    learning_rate=lr_ft[i],
#                    batch_size=16,
#                    early_stopping=5,
#                    metrics='none',
#                    split='random',
#                    save_path='./split_learning_rate_'+str(lr_ft[i]),
#                   )
    
#     clf.fit("./mol_train_split.csv") # 训练模型 

### 5.4. “MolPredict”模块的参数说明 
#### load_model:训练好的模型路径 

In [ ]:
# 使用手动拆分的数据valid_data_split验证模型的性能 
# threshold = 0.5

# valid_split_results = {} 

# lr_ft = [1e-5, 1e-4, 1e-3] # 学习率
# for i in range(len(lr_ft)): # 循环输入每个学习率 
#     valid_pred_model = MolPredict(load_model='./split_learning_rate_'+str(lr_ft[i])) # 载入不同学习率下的模型 
    
#     valid_pred = valid_pred_model.predict("./mol_valid_split.csv") #对测试数据进行预测 
    
#     valid_results = pd.DataFrame({'pred':valid_pred.reshape(-1), 
#                            'SMILES':valid_data_split["SMILES"], 
#                            'Target_BBB':valid_data_split["TARGET"]}) 
    
#     auc = roc_auc_score(valid_results.Target_BBB, valid_results.pred) 
#     fpr, tpr, _ = roc_curve(valid_results.Target_BBB, valid_results.pred) 
#     f2_score = fbeta_score(
#         valid_results.Target_BBB, 
#         [1 if p > threshold else 0 for p in valid_results.pred], 
#         beta=2
#     )
    
#     valid_split_results[f"Split Learning Rate: {lr_ft[i]}"] = {"AUC": auc, "FPR": fpr, "TPR": tpr, "F2_Score": f2_score} 
#     print(f"[Learning Rate: {lr_ft[i]}]\tAUC:{auc:.4f}\tF2_Score:{f2_score:.4f}") 

# sorted_valid_split_results = sorted(valid_split_results.items(), key=lambda x: x[1]["F2_Score"], reverse=True) # 根据AUC对结果进行排序

# # 绘制ROC曲线
# plt.figure(figsize=(10, 6), dpi=200)
# plt.title("ROC Curves")
# plt.xlabel("False Positive Rate")
# plt.ylabel("True Positive Rate")

# for name, result in sorted_valid_split_results:
#     if name.startswith("Split Learning Rate"):
#         plt.plot(result["FPR"], result["TPR"], label=f"{name} (AUC:{result['AUC']:.4f} F2_Score:{result['F2_Score']:.4f})")

# plt.legend(loc="lower right") 
# plt.title("Validation_Split_Model", fontdict=font) 
# plt.show() 

## 6. 使用效果最好的模型对测试数据进行预测 

In [ ]:
# 使用全量数据训练的模型在对test数据进行预测 
threshold = 0.5

test_full_pred_model = MolPredict(load_model='./full_learning_rate_'+str(1e-4)) # 载入不同学习率下的模型 

test_smi = {"SMILES": test_data["SMILES"].values.tolist()} 

test_full_pred = test_full_pred_model.predict(test_smi["SMILES"]) #对测试数据进行预测 
    
test_full_results = pd.DataFrame({'pred':test_full_pred.reshape(-1), 
                                  'SMILES':test_data["SMILES"]}) 

test_full_results["TARGET"] = [1 if x > threshold else 0 for x in test_full_results["pred"].values.tolist()] 

test_full_results[["SMILES", "TARGET"]].to_csv('submission.csv', index=False, header=True) 

In [ ]:
# 使用手动拆分的数据训练的模型对test数据进行预测 
# test_split_pred_model = MolPredict(load_model='./split_learning_rate_'+str(1e-3)) # 载入不同学习率下的模型 

# test_split_pred = test_split_pred_model.predict(test_smi["SMILES"]) #对测试数据进行预测 
    
# test_split_results = pd.DataFrame({'pred':test_split_pred.reshape(-1), 
#                                    'SMILES':test_data["SMILES"]}) 

# test_split_results["TARGET"] = [1 if x > threshold else 0 for x in test_split_pred["pred"].values.tolist()] 

# test_split_results[["SMILES", "TARGET"]].to_csv('./Split_Model_Submission.csv', index=False, header=True) 